# Home Assignment 5, Yanal Yahya, 197 group, SE

## Problem 1 (3 points). Shamir secret sharing

Find the secret and initial polynomial in (n, 4)-threshold Shamir secret sharing scheme over $F_{19}$ for $(3,17); (17,5); (5,11); (11,4)$. Is it consistnet with $(4, 3)$?

In [71]:
from sympy.interactive import init_printing
import sympy as sy

In [72]:
init_printing(pretty_print=True) 

In [73]:
p0 = [3, 17]
p1 = [17, 5]
p2 = [5, 11]
p3 = [11, 4]
x, p = sy.symbols('x p')

l0 = sy.simplify(((x - p1[0])/(p0[0] - p1[0]))*((x - p2[0])/(p0[0] - p2[0]))*((x - p3[0])/(p0[0] - p3[0])))
l1 = sy.simplify(((x - p0[0])/(p1[0] - p0[0]))*((x - p2[0])/(p1[0] - p2[0]))*((x - p3[0])/(p1[0] - p3[0])))
l2 = sy.simplify(((x - p0[0])/(p2[0] - p0[0]))*((x - p1[0])/(p2[0] - p1[0]))*((x - p3[0])/(p2[0] - p3[0])))
l3 = sy.simplify(((x - p0[0])/(p3[0] - p0[0]))*((x - p1[0])/(p3[0] - p1[0]))*((x - p2[0])/(p3[0] - p2[0])))
l0, l1, l2, l3

In [74]:
fx = sy.Mod(sy.simplify(p0[1] * l0 + p1[1] * l1 + p2[1] * l2 + p3[1] * l3), p)
print('polynomial:')
fx

polynomial:


⎛      3        2                  ⎞      
⎜  17⋅x    785⋅x    11495⋅x   20717⎟      
⎜- ───── + ────── - ─────── + ─────⎟ mod p
⎝   2016    2016      2016     672 ⎠      

In [76]:
print('secret:')
f0 = fx.subs({x:0, p:19})
f0

secret:


In [82]:
print('Is it consistnet with  (4,3)?')

Is it consistnet with  (4,3)?


In [90]:
fx.subs({x:4, p:19})

The answer is: **NO**

## Problem 2 (2* points). Schnorr ring signature

For the elliptic curve $y^2=x^3+7$ over $F_{127}$ and the hash function $H() = SHA256()\%127$. Consider a subgroup of point on a curve with a generator $G = (19, 32)$.

Suppose Alice has a pricte key $a = 17$ and Bob has a prite key $b = 29$.

1. (0 points) Compute public keys.
2. (2* points) Generate Schnorr's ring signature for $m=77$ on Alice's behalf with $k=101$.

In [98]:
G = [19, 32]
a = 17
b = 29
m = 77
k = 101

In [99]:
# 1. (0 points) Compute public keys.
publicA = [G[0] * a , G[1] * a]
publicB = [G[0] * b , G[1] * b]
publicA, publicB

In [100]:
#2. (2* points) Generate Schnorr's ring signature for $m=77$ on Alice's behalf with $k=101$.


## Problem 3 (3 points). Bitcoin data

Consider tx=21e8fd77b81b6c4cf6723c10ea83acae77728f188ee6f4cd97ae114665b5fc0f (block height: 478994, tx index/position: 1603). Trace the transaction back to address 12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw.

Use blockexplorer Python library, Bitcoin RPC or their equivalent alternative.

In [115]:
from blockchain import blockexplorer

In [236]:
def get_compressed_form_of_public_key(uncompressed_pubkey):
    x = uncompressed_pubkey[0:(int(len(uncompressed_pubkey) / 2))]
    y = uncompressed_pubkey[(int(len(uncompressed_pubkey) / 2)):]
    if int(y, 16) % 2 == 0:
        pubkey = '2' + x
    else:
        pubkey = '3' + x
    return pubkey

# get public keys compressed
def get_pubkeysc(pubkeys):
    pubkeysc = []
    for pubkey in pubkeys:
        pubkeyc = get_compressed_form_of_public_key(pubkey)
        pubkeysc.append(pubkeyc)
    return pubkeysc

def get_hashes_hex(deserialized_script):
    hashes = deserialized_script['hashes']
    hashes_hex = []
    for hash in hashes:
        hashes_hex.append(hash.hex())
    return hashes_hex

def get_pubkeys_hex(deserialized_script):
    pubkeys = deserialized_script['keys']
    pubkeys_hex = []
    for pubkey in pubkeys:
        pubkeys_hex.append(pubkey.hex())
    return pubkeys_hex

from bitcoinaddress import Address
from bitcoinaddress.util import hash160, ecdsa_secp256k1, checksum
from bitcoinaddress import segwit_addr

# SEGWIT_PREFIX = b'\x05'
# P2SH_P2WPKH
def generate_public_address_3(pubkeyc, prefix= b'\x05'):
    hex_str = bytearray.fromhex(pubkeyc)
    prefix_redeem = b'\x00\x14'
    redeem_script = hash160(prefix_redeem + hash160(hex_str).digest()).digest()  # 20 bytes
    m = prefix + redeem_script
    c = checksum(m)
    return base58.b58encode(m + c).decode('utf-8')

def get_public_addresses_3(pubkeysc):
    pub_addresses_3 = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_3(pubkeyc)
        pub_addresses_3.append(adr)
    return pub_addresses_3

def generate_public_address_1_compressed(pubkeyc):
    hex_str = bytearray.fromhex(pubkeyc)
    key_hash = '00' + hash160(hex_str).hexdigest()
    sha = hashlib.sha256()    
    sha.update(bytearray.fromhex(key_hash))
    checksum = sha.digest()
    sha = hashlib.sha256()
    sha.update(checksum)
    checksum = sha.hexdigest()[0:8]
    return base58.b58encode( bytes(bytearray.fromhex(key_hash + checksum))).decode('utf-8')

def get_public_addresses_1_compressed(pubkeysc):
    pub_addresses_1_c = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_1_compressed(pubkeyc)
        pub_addresses_1_c.append(adr)
    return pub_addresses_1_c

def generate_public_address_bech32_P2WPKH(pubkeyc, bech32_prefix=Address.MainNet.BECH32_PREFIX):
    hex_str = bytearray.fromhex(pubkeyc)
    redeem_script_P2WSH = hash160(hex_str).digest()
    adr =  str(segwit_addr.encode(bech32_prefix, Address.WITNESS_VERSION, redeem_script_P2WSH))
    return adr

def get_public_addresses_bech32_P2WPKH(pubkeysc):
    addresses = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_bech32_P2WPKH(pubkeyc)
        addresses.append(adr)
    return addresses

def generate_public_address_bech32_P2WSH(pubkeyc, bech32_prefix=Address.MainNet.BECH32_PREFIX):
    hex_str = bytearray.fromhex(pubkeyc)
    redeem_script_P2WSH = hash160(hex_str).digest()
    adr =  str(segwit_addr.encode(bech32_prefix, Address.WITNESS_VERSION, redeem_script_P2WSH))
    return adr

def get_public_addresses_bech32_P2WSH(pubkeysc):
    addresses = []
    for pubkeyc in pubkeysc:
        adr = generate_public_address_bech32_P2WSH(pubkeyc)
        addresses.append(adr)
    return addresses

from bitcoinlib.transactions import script_deserialize

def generate_outputs_dict(outputs):
    shortened_outputs = {}
    for output in outputs:
        deserialized_script = script_deserialize(output.script)
        # hash160_pubkeys = get_hashes_hex(deserialized_script)

        pubkeys_hex = get_pubkeys_hex(deserialized_script)
        pubkeysc_hex = get_pubkeysc(pubkeys_hex)
        public_addresses_3 = get_public_addresses_3(pubkeysc_hex)
        public_addresses_1_compressed = get_public_addresses_1_compressed(pubkeysc_hex)
        public_addresses_bc1_p2wpkh = get_public_addresses_bech32_P2WPKH(pubkeysc_hex)
        public_addresses_bc1_p2wsh = get_public_addresses_bech32_P2WSH(pubkeysc_hex)

        shortened_outputs.update({
            output.n: {
                "pubkeys": pubkeys_hex,
                "pubkeysc": pubkeysc_hex,
                "address": output.address,
                "address 1 compressed": public_addresses_1_compressed,
                "address 3": public_addresses_3,
                "address bc1 P2WPKH ": public_addresses_bc1_p2wpkh,
                "address bc1 P2WSH ": public_addresses_bc1_p2wsh,
                "locking_script_type": deserialized_script['script_type'],
                "script": output.script,
                "value": output.value,
                "m": deserialized_script['number_of_sigs_m'],
                "n": deserialized_script['number_of_sigs_n']
            }
        })

    return shortened_outputs

def generate_inputs_dict(inputs):
    shortened_inputs = {}
    for id_input in range(len(inputs)):
        input = inputs[id_input]
        try:
            if input.value == 1215261 or input.value == 315637178:
                print(dir(input))
                print(input.script)
                print(input.script_sig)
                print(input.type)
                print(input.address)
                print(input.value)
                print(input.tx_index)
                print(input.sequence)
                print("-------------------------")
        except:
            pass
        try:
            deserialized_script_sig = script_deserialize(input.script_sig)
            deserialized_script = script_deserialize(input.script)

            shortened_inputs.update({
                id_input: {
                    "pubkeys": get_pubkeys_hex(deserialized_script_sig),
                    "address when it was output": input.address,
                    "script": input.script,
                    "script_type": deserialized_script['script_type'] + '_' + deserialized_script_sig['script_type'],
                    "value": input.value,
                    "script_sig": input.script_sig,
                    "m": deserialized_script_sig['number_of_sigs_m'],
                    "n": deserialized_script_sig['number_of_sigs_n'],
                    "tx_index": input.tx_index,
                    
                }
            })

        except:
            # coinbase transaction
            pass
    return shortened_inputs

In [232]:
def get_block(block_id):
    return blockexplorer.get_block(block_id)

# loading block
block = get_block(block_id="478994")

In [195]:
tx = block.transactions[1603]

In [238]:
def traceAddress(transaction, address = '12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw'):
    t = ''
    inputs = generate_inputs_dict(transaction.inputs)
    for inpt in inputs:
        inpt = inputs[inpt]
        if(inpt['address when it was output'] == address):
            return {"hash": transaction.hash,
                            "inputs": generate_inputs_dict(transaction.inputs),
                            "outputs": generate_outputs_dict(transaction.outputs),
                            "block_height": transaction.block_height
                           }
    for inpt in inputs:
        inpt = inputs[inpt]
        return traceAddress(transaction = blockexplorer.get_tx(str(inpt['tx_index'])))

In [ ]:
result = traceAddress(transaction = block.transactions[1603])

In [249]:
result

{'hash': '409803bb5e124fd028c0482027c7722e84ce55b78204b279d3a44aba5e7c1698',
 'inputs': {0: {'pubkeys': ['026eace100993a8d73f3ce1761fcfd359c015b04a9b9ebbcf9a33e9a9ca81e9c2f'],
   'address when it was output': '12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw',
   'script': '76a91414a477964ed719135d1598da348a858b18b44fd588ac',
   'script_type': 'p2pkh_sig_pubkey',
   'value': 17124673,
   'script_sig': '483045022100f466e70d53728e4cd222367f9236fbed484590bec57bae4a46898379bc6c34b8022053d8a12c030c7ae941ec46af7b02b32e6ea1493bf5fe83c017c38795a8a9e79b0121026eace100993a8d73f3ce1761fcfd359c015b04a9b9ebbcf9a33e9a9ca81e9c2f',
   'm': 1,
   'n': 1,
   'tx_index': 5142685607303839},
  1: {'pubkeys': ['026eace100993a8d73f3ce1761fcfd359c015b04a9b9ebbcf9a33e9a9ca81e9c2f'],
   'address when it was output': '12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw',
   'script': '76a91414a477964ed719135d1598da348a858b18b44fd588ac',
   'script_type': 'p2pkh_sig_pubkey',
   'value': 18010230,
   'script_sig': '483045022100d0b9a462e1e7d7208f

## Problem 4 (3* points). Shared send untangling

1. (1* point) Simplify transaction (see Section 1 \[1\])
* 409803bb5e124fd028c0482027c7722e84ce55b78204b279d3a44aba5e7c1698
* 9ea6fdad2ea5870f5b17cb28587263f90cd7ec5c67f35d395a318186e5c79333
* c38aac9910f327700e0f199972eed8ea7c6b1920e965f9cb48a92973e7325046
* 659135664894e50040830edb516a76f704fd2be409ecd8d1ea9916c002ab28a2
2. (2* points) What are these transactions types (see Theorem 1)?

\[1\] https://bitfury.com/content/downloads/bitfury_whitepaper_shared_send_untangling_in_bitcoin_8_24_2016.pdf

* **409803bb5e124fd028c0482027c7722e84ce55b78204b279d3a44aba5e7c1698**<br>
```1. Tx has 36 inputs with the same address and 2 different outputs not equal to any of the inputs, this means that to simplify, we simply sum the balance of all inputs and replace the input by one input address(as the 36 addresses are the same) with the balance of the calculated sum.```<br>
```2. Transaction type: simple```
* **9ea6fdad2ea5870f5b17cb28587263f90cd7ec5c67f35d395a318186e5c79333**<br>
```1. Nothing to simplify here, the inputs and outputs contains of 4 different addresses)```<br>
```2. Transaction type: Simple```
* **c38aac9910f327700e0f199972eed8ea7c6b1920e965f9cb48a92973e7325046**<br>
```1. Nothing to simplify here, the inputs and outputs contains of 6 different addresses(2 inputs and 4 putputs))```<br>
```2. Transaction type: Ambiguous```
* **659135664894e50040830edb516a76f704fd2be409ecd8d1ea9916c002ab28a2**<br>
```1. Firstly the inputs and outputs Don't cross each other, so we will only simplify the inputs. I did that by scripts below```<br>
```2. Transaction type: Ambiguous```

In [251]:
tx4 = blockexplorer.get_tx('659135664894e50040830edb516a76f704fd2be409ecd8d1ea9916c002ab28a2')

In [ ]:
inputs4 = generate_inputs_dict(tx4.inputs)

In [278]:
inpts = {}
for inpt in inputs4:
    inpt = inputs4[inpt]
    key = inpt['address when it was output']
    if(key in inpts):
        inpts[key] = {'count': inpts[key]['count'] + 1, 'value': inpts[key]['value'] + inpt['value']}
    else:
        inpts[inpt['address when it was output']] = {'count': 1, 'value': inpt['value']} 

In [279]:
inpts, len(inpts)

({'1K1aX96a9GmXJFtwbzjhp7ToDzHoPjtzv2': {'count': 8, 'value': 14400},
  '14HzRFwHQqXr9GP44RvWeSoas9g4KvmGJK': {'count': 20, 'value': 41280},
  '1JZLRuvrk4KkmGbRwZ4xMy5SwL3wFgjHL4': {'count': 20, 'value': 46720},
  '1C41rAii831NvtkEonnG1NfZcfyVh1akAS': {'count': 17, 'value': 50800},
  '1LPc6C2GU4roiRjoie8Aav9vguYLZVy9F1': {'count': 11, 'value': 25040},
  '1LrVkgi5Ub9MwVFscHrT4fdMD8bBPcuz73': {'count': 24, 'value': 69520},
  '1Jgw1tMafL38MJ2Cmj75Qfg8XvUjR6afpV': {'count': 39, 'value': 108160},
  '1QELuJoy8w5MFA4nszaGd6sZ6iFf3MMatt': {'count': 34, 'value': 86320},
  '128doqZjtivEQTb5ynGWeoFaGqmLJqvsHh': {'count': 7, 'value': 12080},
  '1NKmtCdN5nGT1zMeUtogpDYDWLceqjSsBQ': {'count': 29, 'value': 70800},
  '1Fm1pLaEFRWJ7EMVN6ZRwK359BTVX2VZgM': {'count': 11, 'value': 25040},
  '1AAns1kEaZgC479cNXPda5rUYXsH5Ya9KR': {'count': 16, 'value': 27200},
  '17gyVwL197EK9vhcCM7a91LzfnvQSyU4Z7': {'count': 12, 'value': 20880},
  '1Kppd36Qy28PBpyy38HbCzESzD4A5YuCLD': {'count': 36, 'value': 96480},
  '1hUi

## Problem 5 (4 points). ERC20 data

Consider (scam?) token 0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0 from Ethereum mainnet. How many tokens where transfered in Dec-2020?

In [299]:
import requests, json
from web3 import Web3, HTTPProvider

In [300]:
web3_url = "https://mainnet.infura.io/v3/f25af56bef3b4460a475fd5ddc05b227"
w3 = Web3(Web3.HTTPProvider(web3_url))

In [301]:
# scam contract from https://raw.githubusercontent.com/MyEtherWallet/ethereum-lists/master/dist/tokens/eth/tokens-eth.json

address = '0xCae72A7A0Fd9046cf6b165CA54c9e3a3872109E0'
# ABI (Application Binary Interface)
abi = '[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"burn","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"isOwner","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"mint","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"}]'
contract_instance = w3.eth.contract(address=address, abi=abi)

In [302]:
print(contract_instance.functions.name().call())

AnRKey X


In [311]:
# import dependencies
import pickle
from tqdm import tqdm

ending_blocknumber = 11477000 # w3.eth.blockNumber

starting_blocknumber = 11476000 

# filter through blocks and look for transactions involving this address
blockchain_address = '0xCae72A7A0Fd9046cf6b165CA54c9e3a3872109E0'

# create an empty dictionary we will add transaction data to
tx_dictionary = {}

def getTransactions(start, end, address):
    '''This function takes three inputs, a starting block number, ending block number
    and an Ethereum address. The function loops over the transactions in each block and
    checks if the address in the to field matches the one we set in the blockchain_address.
    Additionally, it will write the found transactions to a pickle file for quickly serializing and de-serializing
    a Python object.'''
    print(f"Started filtering through block number {start} to {end} for transactions involving the address - {address}...")
    for x in tqdm(range(start, end)):
        block = w3.eth.getBlock(x, True)
        for transaction in block.transactions:
            if transaction['to'] == address or transaction['from'] == address:
                with open("transactions.pkl", "wb") as f:
                    hashStr = transaction['hash'].hex()
                    tx_dictionary[hashStr] = transaction
                    pickle.dump(tx_dictionary, f)
                f.close()
    print(f"Finished searching blocks {start} through {end} and found {len(tx_dictionary)} transactions")
    return tx_dictionary
    

tx_dictionary = getTransactions(starting_blocknumber, ending_blocknumber, blockchain_address)

Started filtering through block number 11476000 to 11477000 for transactions involving the address - 0xCae72A7A0Fd9046cf6b165CA54c9e3a3872109E0...


100%|███████████████████████████████████████| 1000/1000 [04:48<00:00,  3.46it/s]

Finished searching blocks 11476000 through 11477000 and found 724 transactions


In [312]:
tx_dictionary

{'0xac20afb3319151ff202ec516d67ddc025e5c3ca02b582c547ee10e945fb722c2': AttributeDict({'blockHash': HexBytes('0xa205c6fe1a3441ac80054536d4e77bc7da9b55291cbcd17cc028afc14763fa9e'),
  'blockNumber': 11476007,
  'chainId': '0x1',
  'from': '0x63A64ECF0a751288E54aEe7356842E448ea9485C',
  'gas': 66754,
  'gasPrice': 69000000000,
  'hash': HexBytes('0xac20afb3319151ff202ec516d67ddc025e5c3ca02b582c547ee10e945fb722c2'),
  'input': '0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff',
  'nonce': 626,
  'r': HexBytes('0x0eca571ab5a325aaf5de90989e9bb7150b0d6d03adb0b645e9805eeecedb5e3f'),
  's': HexBytes('0x681a172b53ca03539f636f0f1a0225967475bafdd23dd2238cc896d9f5dee8ea'),
  'to': '0xCae72A7A0Fd9046cf6b165CA54c9e3a3872109E0',
  'transactionIndex': 18,
  'type': '0x0',
  'v': 38,
  'value': 0}),
 '0x9751a18cff66a334b90f63e08ca06a141f2a0e9249033bc84bb046da118531a5': AttributeDict({'blockHash': HexBytes('0x3836521

Next we just filter this dictionary, so we get the transactions which have 'transfer' method and not failed and which date is in december 2020
I just did't know where to find the date of tx and its method and fail status (((
but the code next should be something like this:

In [ ]:
filterd_txs = {}
for txx in tx_dictionary:
    if(tx_dictionary[txx]['date'] >= "01.12.2020" and tx_dictionary[txx]['date'] <= "31.12.2020" 
       and !tx_dictionary[txx]['is_fail'] and tx_dictionary[txx]['method'] == 'transfer'):
        filterd_txs[txx] = tx_dictionary[txx]
filterd_txs        

In [330]:
#tx = w3.eth.getTransaction('0x5f2ac1b06a0388d70ee928f8f6f4d070777ede336e8ded7c29508a274fd94db0')
#tx

In [332]:
#import json
#import urllib.request
#from web3_input_decoder import decode_constructor, decode_function

#decode_function(json.loads(abi), tx['input'])

[('address', 'spender', '0x7a250d5630b4cf539739df2c5dacb4c659f2488d'),
 ('uint256', 'amount', 79000000000000000000000000000)]

## Problem 6 (3* points). BigQuery

Consider (scam?) token 0xcae72a7a0fd9046cf6b165ca54c9e3a3872109e0 from Ethereum mainnet. Compute the total amount of tokens transfered. Use bigquery-public-data.crypto_ethereum (https://ethereum-etl.readthedocs.io/en/latest/google-bigquery/).